## 创建Regex对象
正则表达式可以很方便地从符合特定结构的字符串中提取数据。

我们用三重双引号来表示正则表达式字符串，否则，就不得不对正则表达式的反斜杠进行转义，例如用`\\s` 表示`\s`。你还可以通过创建一个Regex 类的实例来定义正则表达式，如new Regex("""\W""")，但这种用法并不常见。

scala的scala.util.matching.Regex类封装了java的正则表达式．要构造一个Regex对象，使用String类的r方法即可:

In [1]:
val numPattern = "[0-9]+".r

numPattern: util.matching.Regex = [0-9]+

如果正则表达式包含反斜杠或引号的话，那么最好使用"原始"字符串语法"""...""":

In [5]:
val positiveNumPattern = """^[1-9]\d*$""".r

positiveNumPattern: util.matching.Regex = ^[1-9]\d*$

如果在Java中使用上述正则表达式，则应该使用下面方式(需要进行转义):

In [6]:
val positiveNumPattern = "^[1-9]\\d*$".r

positiveNumPattern: util.matching.Regex = ^[1-9]\d*$

**注意：**在三个双引号内的正则表达式中使用变量插值是无效的。你依然需要对变量插值进行转义，例如，你应该用`s"""$first\\s+$second""".r`，而不是`s"""$first\s+$second""".r`。而如果你没有使用变量插值，则不必转义。

In [10]:
val (first,second) = ("first", "second")

first: String = "first"
second: String = "second"

In [12]:
s"""$first\\s+$second""".r

res11: util.matching.Regex = first\s+second

In [13]:
//会抛出异常
//s"""$first\s+$second""".r

## 正则表达式的模式匹配

在赋值中使用re的模式匹配：

In [19]:
val pattern = "([0-9]+) ([a-zA-Z]+)".r
//必须完全匹配才行
val pattern(count, fruit) = "100 Bananas"

pattern: util.matching.Regex = ([0-9]+) ([a-zA-Z]+)
count: String = "100"
fruit: String = "Bananas"

更一般的re模式匹配：

In [22]:
/*
该正则表达式匹配一个用于表示书本的字符串，其中有两个捕捉组（注意正则表达式中的括号），一个表示标题，一个表示作者。
调用r 方法以创建正则表达式。
第二个正则表达式匹配一个用于表示杂志的字符串，其中的捕捉组表示杂志标题和发行时间。
*/
val BookExtractorRE = """Book: title=([^,]+),\s+author=(.+)""".r
val MagazineExtractorRE = """Magazine: title=([^,]+),\s+issue=(.+)""".r

val catalog = Seq(
  "Book: title=Programming Scala Second Edition, author=Dean Wampler",
  "Magazine: title=The New Yorker, issue=January 2014",
  "11Magazine: title=The New Yorker, issue=January 2014",//也必须是完全匹配
  "Unknown: text=Who put this here??"
)

for (item <- catalog) {
  item match {
    case BookExtractorRE(title, author) =>
      println(s"""Book "$title", written by $author""")
    case MagazineExtractorRE(title, issue) =>
      println(s"""Magazine "$title", issue $issue""")
    case entry => println(s"Unrecognized entry: $entry")
  }
}

Book "Programming Scala Second Edition", written by Dean Wampler
Magazine "The New Yorker", issue January 2014
Unrecognized entry: 11Magazine: title=The New Yorker, issue=January 2014
Unrecognized entry: Unknown: text=Who put this here??


BookExtractorRE: util.matching.Regex = Book: title=([^,]+),\s+author=(.+)
MagazineExtractorRE: util.matching.Regex = Magazine: title=([^,]+),\s+issue=(.+)
catalog: Seq[String] = List(
  "Book: title=Programming Scala Second Edition, author=Dean Wampler",
  "Magazine: title=The New Yorker, issue=January 2014",
  "11Magazine: title=The New Yorker, issue=January 2014",
  "Unknown: text=Who put this here??"
)

如果想从多个匹配项中提取分组内容，可以使用如下命令:

In [24]:
val str = "3 a b c 4 f"
val numPattern = "([0-9]+) ([a-z]+)".r
for(numPattern(num, letter) <- numPattern.findAllIn(str)){
    println(num + "---"+letter)
}

for(numPattern(num, letter) <- numPattern.findAllMatchIn(str)){
    println(num + "---"+letter)
}

3---a
4---f
3---a
4---f


str: String = "3 a b c 4 f"
numPattern: util.matching.Regex = ([0-9]+) ([a-z]+)

In [73]:
val madhatter = "(h)(?=(at[^a]+))".r
val madhats   = (madhatter findAllMatchIn hathaway map {
    case madhatter(x,y) => s"$x$y"
}).toList     // List(hath, hatth, hattth, hatttt)

madhatter: Regex = (h)(?=(at[^a]+))
madhats: List[String] = List("hath", "hatth", "hattth", "hatttt")

In [74]:
import scala.util.matching.Regex
val datePattern = new Regex("""(\d\d\d\d)-(\d\d)-(\d\d)""", "year", "month", "day")
val text = "From 2011-07-15 to 2011-07-17"
val repl = datePattern replaceAllIn (text, m => s"${m group "month"}/${m group "day"}")

import scala.util.matching.Regex

datePattern: Regex = (\d\d\d\d)-(\d\d)-(\d\d)
text: String = "From 2011-07-15 to 2011-07-17"
repl: String = "From 07/15 to 07/17"

## Regex常用方法

### findFirstIn与findFirstMatchIn
方法findFirstIn可以匹配到第一个符合正则表达式规则的字符串，没有匹配到任何结果的情况下返回None。

```scala
  def findFirstIn(source: CharSequence): Option[String] = {
    val m = pattern.matcher(source)
    if (m.find) Some(m.group) else None
  }

  def findFirstMatchIn(source: CharSequence): Option[Match] = {
    val m = pattern.matcher(source)
    if (m.find) Some(new Match(source, m, groupNames)) else None
  }
```

先看看官方代码的事例：

In [45]:
"""\w+""".r findFirstIn "A simple example." foreach println // prints "A"

A


In [46]:
("""[a-z]""".r findFirstMatchIn "A simple example.") map (_.start) 

res45: Option[Int] = Some(2)

自己测试一下：

In [25]:
val str = "aaa1122ff3344gg5566"
val numPattern = "[0-9]+".r

str: String = "aaa1122ff3344gg5566"
numPattern: util.matching.Regex = [0-9]+

In [34]:
val matchOption = numPattern.findFirstIn(str)

matchOption: Option[String] = Some("1122")

In [36]:
//此函数不能带参数
matchOption.get

res35: String = "1122"

In [38]:
val matchOption = numPattern.findFirstMatchIn(str)

matchOption: Option[util.matching.Regex.Match] = Some(1122)

In [39]:
val matchRst = matchOption.get

matchRst: util.matching.Regex.Match = 1122

In [40]:
matchRst.matched

res39: String = "1122"

In [41]:
matchRst.group(0)

res40: String = "1122"

In [43]:
matchRst.end

res42: Int = 7

### findAllIn与findAllMatchIn
方法findAllIn可以匹配到所有符合正则表达式规则的字符串，返回的结果是一个Iterator。你可以把它转化为一个Array来使用。如果没有匹配到任何符合规则的字符换那么，它将返回一个空Iterator。
```scala
  def findAllIn(source: CharSequence) = new Regex.MatchIterator(source, this, groupNames)

  def findAllMatchIn(source: CharSequence): Iterator[Match]
```

先看看官方代码的事例：

In [47]:
val hat  = "hat[^a]+".r
val hathaway = "hathatthattthatttt"
val hats = (hat findAllIn hathaway).toList                     // List(hath, hattth)
val pos  = (hat findAllMatchIn hathaway map (_.start)).toList  // List(0, 7)

hat: util.matching.Regex = hat[^a]+
hathaway: String = "hathatthattthatttt"
hats: List[String] = List("hath", "hattth")
pos: List[Int] = List(0, 7)

In [48]:
val madhatter = "(h)(?=(at[^a]+))".r
val madhats   = (madhatter findAllMatchIn hathaway map {
    case madhatter(x,y) => s"$x$y"
}).toList     // List(hath, hatth, hattth, hatttt)

madhatter: util.matching.Regex = (h)(?=(at[^a]+))
madhats: List[String] = List("hath", "hatth", "hattth", "hatttt")

In [49]:
for (words <- """\w+""".r findAllIn "A simple example.") yield words

res48: Iterator[String] = non-empty iterator

In [51]:
(for (words <- """\w+""".r findAllIn "A simple example.") yield words) toList

res50: List[String] = List("A", "simple", "example")

In [52]:
for (words <- """\w+""".r findAllMatchIn "A simple example.") yield words.start

res51: Iterator[Int] = non-empty iterator

In [54]:
(for (words <- """\w+""".r findAllMatchIn "A simple example.") yield words.start).toList

res53: List[Int] = List(0, 2, 9)

自己测试一下：

In [55]:
val str = "aaa1122ff3344gg5566"
val numPattern = "[0-9]+".r

str: String = "aaa1122ff3344gg5566"
numPattern: util.matching.Regex = [0-9]+

In [57]:
numPattern.findAllIn(str).toList

res56: List[String] = List("1122", "3344", "5566")

In [58]:
numPattern.findAllMatchIn(str).toList

res57: List[util.matching.Regex.Match] = List(1122, 3344, 5566)

In [60]:
numPattern.findAllMatchIn(str).map(_.matched).toList

res59: List[String] = List("1122", "3344", "5566")

### findPrefixOf与findPrefixMatchOf
和findFirstIn不同，此方法从字符串的开头开始匹配，和python re的match方法类似
```scala
  def findPrefixOf(source: CharSequence): Option[String] = {
    val m = pattern.matcher(source)
    if (m.lookingAt) Some(m.group) else None
  }

  def findPrefixMatchOf(source: CharSequence): Option[Match] = {
    val m = pattern.matcher(source)
    if (m.lookingAt) Some(new Match(source, m, groupNames)) else None
  }
```

In [61]:
// returns None, since the text does not begin with a lowercase letter
"""\p{Lower}""".r findPrefixOf "A simple example."

res60: Option[String] = None

In [62]:
// returns Some(" simple example.")
"""\w+""".r findPrefixMatchOf "A simple example." map (_.after) 

res61: Option[CharSequence] = Some( simple example.)

In [63]:
"""\w+""".r findPrefixMatchOf "A simple example."

res62: Option[util.matching.Regex.Match] = Some(A)

### replaceFirstIn、replaceSomeIn、replaceAllIn
用于替换字符串
```scala
  def replaceAllIn(target: CharSequence, replacement: String): String

  def replaceAllIn(target: CharSequence, replacer: Match => String): String

  def replaceSomeIn(target: CharSequence, replacer: Match => Option[String]): String

  def replaceFirstIn(target: CharSequence, replacement: String): String
```

我们可以调用字符串的replaceAll方法来进行字符换替换，这个方法接收两个参数，第一个是正则表达，第二个是要被替换成的字符串。

In [64]:
"123 Main Street".replaceAll("[0-9]", "x")

res63: String = "xxx Main Street"

还有一种方式是，提前编译好一个正则表达式，然后调用正则表达式的replaceAllIn方法来进行替换。

In [65]:
"[0-9]".r.replaceAllIn("123 Main Street", "x")

res64: String = "xxx Main Street"

如果你只想替换第一次出现的位置的字符换，而不是所有匹配的位置，那么你可以使用字符串的replaceFirst或者正则表达式的replaceFirstIn方法。

In [66]:
"123".replaceFirst("[0-9]", "x")

res65: String = "x23"

In [67]:
"[0-9]".r.replaceFirstIn("123","x")

res66: String = "x23"

来看看官方代码更为灵活的替换：

In [68]:
import util.matching.Regex.{Match,quoteReplacement}
val vars = Map("x" -> "a var", "y" -> """some $ and \ signs""")
val text = "A text with variables %x, %y and %z."
val varPattern = """%(\w+)""".r
val mapper = (m: Match) => vars get (m group 1) map (quoteReplacement(_))
val repl = varPattern replaceSomeIn (text, mapper)

import util.matching.Regex.{Match,quoteReplacement}

vars: Map[String, String] = Map(
  "x" -> "a var",
  "y" -> """
some $ and \ signs
  """
)
text: String = "A text with variables %x, %y and %z."
varPattern: util.matching.Regex = %(\w+)
mapper: util.matching.Regex.Match => Option[String] = <function1>
repl: String = """
A text with variables a var, some $ and \ signs and %z.
"""

In [69]:
vars get ("x") map (quoteReplacement(_))

res68: Option[String] = Some("a var")

In [70]:
vars get ("x")

res69: Option[String] = Some("a var")

In [71]:
val mapper = (m: Match) => vars get (m group 1) map (quoteReplacement(_))
val repl = varPattern replaceSomeIn (text, mapper)

mapper: Match => Option[String] = <function1>
repl: String = """
A text with variables a var, some $ and \ signs and %z.
"""

In [72]:
import scala.util.matching.Regex
val datePattern = new Regex("""(\d\d\d\d)-(\d\d)-(\d\d)""", "year", "month", "day")
val text = "From 2011-07-15 to 2011-07-17"
val repl = datePattern replaceAllIn (text, m => s"${m group "month"}/${m group "day"}")

import scala.util.matching.Regex

datePattern: util.matching.Regex = (\d\d\d\d)-(\d\d)-(\d\d)
text: String = "From 2011-07-15 to 2011-07-17"
repl: String = "From 07/15 to 07/17"

### Regex常用方法源码

#### findFirstIn与findFirstMatchIn
方法findFirstIn可以匹配到第一个符合正则表达式规则的字符串，没有匹配到任何结果的情况下返回None。

```scala
  /** Return an optional first matching string of this `Regex` in the given character sequence,
   *  or None if there is no match.
   *
   *  @param source The text to match against.
   *  @return       An [[scala.Option]] of the first matching string in the text.
   *  @example      {{{"""\w+""".r findFirstIn "A simple example." foreach println // prints "A"}}}
   */
  def findFirstIn(source: CharSequence): Option[String] = {
    val m = pattern.matcher(source)
    if (m.find) Some(m.group) else None
  }

  /** Return an optional first match of this `Regex` in the given character sequence,
   *  or None if it does not exist.
   *
   *  If the match is successful, the [[scala.util.matching.Regex.Match]] can be queried for
   *  more data.
   *
   *  @param source The text to match against.
   *  @return       A [[scala.Option]] of [[scala.util.matching.Regex.Match]] of the first matching string in the text.
   *  @example      {{{("""[a-z]""".r findFirstMatchIn "A simple example.") map (_.start) // returns Some(2), the index of the first match in the text}}}
   */
  def findFirstMatchIn(source: CharSequence): Option[Match] = {
    val m = pattern.matcher(source)
    if (m.find) Some(new Match(source, m, groupNames)) else None
  }
```

#### findAllIn与findAllMatchIn
方法findAllIn可以匹配到所有符合正则表达式规则的字符串，返回的结果是一个Iterator。你可以把它转化为一个Array来使用。如果没有匹配到任何符合规则的字符换那么，它将返回一个空Iterator。
```scala
  /** Return all non-overlapping matches of this `Regex` in the given character 
   *  sequence as a [[scala.util.matching.Regex.MatchIterator]],
   *  which is a special [[scala.collection.Iterator]] that returns the
   *  matched strings but can also be queried for more data about the last match,
   *  such as capturing groups and start position.
   * 
   *  A `MatchIterator` can also be converted into an iterator
   *  that returns objects of type [[scala.util.matching.Regex.Match]],
   *  such as is normally returned by `findAllMatchIn`.
   * 
   *  Where potential matches overlap, the first possible match is returned,
   *  followed by the next match that follows the input consumed by the
   *  first match:
   *
   *  {{{
   *  val hat  = "hat[^a]+".r
   *  val hathaway = "hathatthattthatttt"
   *  val hats = (hat findAllIn hathaway).toList                     // List(hath, hattth)
   *  val pos  = (hat findAllMatchIn hathaway map (_.start)).toList  // List(0, 7)
   *  }}}
   *
   *  To return overlapping matches, it is possible to formulate a regular expression
   *  with lookahead (`?=`) that does not consume the overlapping region.
   *
   *  {{{
   *  val madhatter = "(h)(?=(at[^a]+))".r
   *  val madhats   = (madhatter findAllMatchIn hathaway map {
   *    case madhatter(x,y) => s"$x$y"
   *  }).toList                                       // List(hath, hatth, hattth, hatttt)
   *  }}}
   *
   *  Attempting to retrieve match information before performing the first match
   *  or after exhausting the iterator results in [[java.lang.IllegalStateException]].
   *  See [[scala.util.matching.Regex.MatchIterator]] for details.
   *
   *  @param source The text to match against.
   *  @return       A [[scala.util.matching.Regex.MatchIterator]] of matched substrings.
   *  @example      {{{for (words <- """\w+""".r findAllIn "A simple example.") yield words}}}
   */
  def findAllIn(source: CharSequence) = new Regex.MatchIterator(source, this, groupNames)

  /** Return all non-overlapping matches of this regexp in given character sequence as a
   *  [[scala.collection.Iterator]] of [[scala.util.matching.Regex.Match]].
   *
   *  @param source The text to match against.
   *  @return       A [[scala.collection.Iterator]] of [[scala.util.matching.Regex.Match]] for all matches.
   *  @example      {{{for (words <- """\w+""".r findAllMatchIn "A simple example.") yield words.start}}}
   */
  def findAllMatchIn(source: CharSequence): Iterator[Match] = {
    val matchIterator = findAllIn(source)
    new Iterator[Match] {
      def hasNext = matchIterator.hasNext
      def next: Match = {
        matchIterator.next()
        new Match(matchIterator.source, matchIterator.matcher, matchIterator.groupNames).force
      }
    }
  }
```

#### findPrefixOf与findPrefixMatchOf
和findFirstIn不同，此方法从字符串的开头开始匹配，和python re的match方法类似
```scala
  /** Return an optional match of this `Regex` at the beginning of the
   *  given character sequence, or None if it matches no prefix
   *  of the character sequence.
   *
   *  Unlike `findFirstIn`, this method will only return a match at
   *  the beginning of the input.
   *
   *  @param source The text to match against.
   *  @return       A [[scala.Option]] of the matched prefix.
   *  @example      {{{"""\p{Lower}""".r findPrefixOf "A simple example." // returns None, since the text does not begin with a lowercase letter}}}
   */
  def findPrefixOf(source: CharSequence): Option[String] = {
    val m = pattern.matcher(source)
    if (m.lookingAt) Some(m.group) else None
  }

  /** Return an optional match of this `Regex` at the beginning of the
   *  given character sequence, or None if it matches no prefix
   *  of the character sequence.
   *
   *  Unlike `findFirstMatchIn`, this method will only return a match at
   *  the beginning of the input.
   *
   *  @param source The text to match against.
   *  @return       A [[scala.Option]] of the [[scala.util.matching.Regex.Match]] of the matched string.
   *  @example      {{{"""\w+""".r findPrefixMatchOf "A simple example." map (_.after) // returns Some(" simple example.")}}}
   */
  def findPrefixMatchOf(source: CharSequence): Option[Match] = {
    val m = pattern.matcher(source)
    if (m.lookingAt) Some(new Match(source, m, groupNames)) else None
  }
```

#### replaceFirstIn、replaceSomeIn、replaceAllIn
用于替换字符串
```scala
  /** Replaces all matches by a string.
   *
   *  $replacementString
   *
   *  @param target      The string to match
   *  @param replacement The string that will replace each match
   *  @return            The resulting string
   *  @example           {{{"""\d+""".r replaceAllIn ("July 15", "<NUMBER>") // returns "July <NUMBER>"}}}
   */
  def replaceAllIn(target: CharSequence, replacement: String): String = {
    val m = pattern.matcher(target)
    m.replaceAll(replacement)
  }

  /**
   * Replaces all matches using a replacer function. The replacer function takes a
   * [[scala.util.matching.Regex.Match]] so that extra information can be obtained
   * from the match. For example:
   *
   * {{{
   * import scala.util.matching.Regex
   * val datePattern = new Regex("""(\d\d\d\d)-(\d\d)-(\d\d)""", "year", "month", "day")
   * val text = "From 2011-07-15 to 2011-07-17"
   * val repl = datePattern replaceAllIn (text, m => s"${m group "month"}/${m group "day"}")
   * }}}
   *
   * $replacementString
   *
   * @param target      The string to match.
   * @param replacer    The function which maps a match to another string.
   * @return            The target string after replacements.
   */
  def replaceAllIn(target: CharSequence, replacer: Match => String): String = {
    val it = new Regex.MatchIterator(target, this, groupNames).replacementData
    it foreach (md => it replace replacer(md))
    it.replaced
  }

  /**
   * Replaces some of the matches using a replacer function that returns an [[scala.Option]].
   * The replacer function takes a [[scala.util.matching.Regex.Match]] so that extra
   * information can be btained from the match. For example:
   *
   * {{{
   * import scala.util.matching.Regex._
   *
   * val vars = Map("x" -> "a var", "y" -> """some $ and \ signs""")
   * val text = "A text with variables %x, %y and %z."
   * val varPattern = """%(\w+)""".r
   * val mapper = (m: Match) => vars get (m group 1) map (quoteReplacement(_))
   * val repl = varPattern replaceSomeIn (text, mapper)
   * }}}
   *
   * $replacementString
   *
   * @param target      The string to match.
   * @param replacer    The function which optionally maps a match to another string.
   * @return            The target string after replacements.
   */
  def replaceSomeIn(target: CharSequence, replacer: Match => Option[String]): String = {
    val it = new Regex.MatchIterator(target, this, groupNames).replacementData
    for (matchdata <- it ; replacement <- replacer(matchdata))
      it replace replacement

    it.replaced
  }

  /** Replaces the first match by a string.
   *
   *  $replacementString
   *
   *  @param target      The string to match
   *  @param replacement The string that will replace the match
   *  @return            The resulting string
   */
  def replaceFirstIn(target: CharSequence, replacement: String): String = {
    val m = pattern.matcher(target)
    m.replaceFirst(replacement)
  }
```